# This program fetch artists from a playlist. Then fetch top 10 tracks of each artist and sort them based on popularity. Pick top 100 tracks and add them to another play list. 

## So you get 100 most popular songs

In [1]:
#Install spotify SDK if not installed
!pip install spotipy


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
# Import required libraries
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials #Required for authenticating with soptify
import pandas as pd  #Will use Panda to do data analysis

In [28]:
# Defining some constants 
client_id = "XXXXXX"
client_password = "XXXXXX"
user_name = "XXXXXX"
from_playlist_id = "XXXXXX"
top_tracks_playlist_id = "XXXXXX"


In [32]:
token = util.prompt_for_user_token(username=user_name, scope = "playlist-modify-public",
                                   client_id=client_id,
                                   client_secret=client_password,
                                   redirect_uri="http://localhost:8080/" )
#Add your credentials to authenticate yourself with Spotify
sp = spotipy.Spotify(auth = token)


True


In [35]:
def get_artists(playlist_id, spotify):   
    if type(spotify) is spotipy.client.Spotify:
        # Fetch Playlist
        playlist_data = spotify.playlist(playlist_id=playlist_id)
        print("Playlist: {}, Owner: {}".format(
            playlist_data['name'],
            playlist_data['owner']['display_name']))
        # Create a unique list of Artists so that it can be used to fetch their popular Track
        # As we may get duplicate Artists from different track hence we need to use SET data structure
        artists_id = set()
        artists_name = set()
        for item in playlist_data['tracks']['items']:
            for artist in item['track']['album']['artists']:
                artists_name.add(artist['name'])
                artists_id.add(artist['id'])
        return (artists_id, artists_name)
    else:
        print("Spotify Object is invalid")

def get_popular_track(artists_id, spotify):
    if type(spotify) is spotipy.client.Spotify:
        # Create Disctionary of top 10 popular tracks of the artists
        popular_tracks = []
        for artist in artists_id:
            top_tracks = spotify.artist_top_tracks(artist_id=artist)
            for track in top_tracks['tracks']:
                uri = track['uri']
                popularity = track['popularity']
                name = track['name']
                track_element = {"uri": uri, "name": name, "popularity": popularity, "artist": artist}
                popular_tracks.append(track_element)
        return popular_tracks
    else:
        print("Spotify Object is invalid")
                  

In [38]:
artist_id, artist_name = get_artists(playlist_id=from_playlist_id, spotify=sp)
print("Following artists fetched \n {}".format(artist_name))

popular_tracks = get_popular_track(artist_id, spotify=sp)

#Create Data Frame so that we can manupilate data
df_popular_tracks = pd.DataFrame.from_dict(popular_tracks)
print(df_popular_tracks.head(10))
# Sort based on popularity and fetch first 100 tracks
top_100 = df_popular_tracks.sort_values(by="popularity", ascending=False).iloc[:100]
# Add the tracks to playlist supplied
sp.playlist_add_items(playlist_id=top_tracks_playlist_id, items = list(top_100['uri']))


Playlist: Starring Emraan Hashmi, Owner: Spotify
Following artists fetched 
 {'Various Artists', 'Arijit Singh', 'Himesh Reshammiya', 'Mithoon', 'Rashid Khan', 'Abhijeet', 'Anu Malik', 'Tanishk Bagchi', 'Sangeet', 'Harshit Saxena', 'Ami Mishra', 'Kunaal-Rangon', 'Vishal Bhardwaj', 'Yuvan Shankar Raja', 'Pritam', 'Siddharth Haldipur', 'Vishal-Shekhar', 'Ankit Tiwari', 'Jeet Gannguli', 'Jubin Nautiyal', 'Sangeet & Siddharth'}
